This notebook is for exploring tensorflow basics; tensors, variables, autograd, graphs/functions, modules/layers/modules

A tensor (tf.constant) is an immutable multi-dimensional array, optimized for GPU processing

A variable (tf.variable) is a mutable tensor

Autograd computes the gradients of variables automatically; you first record operations on a GradientTape. Autograd then allows you to find the gradient of any of the input tensors with respect to a computed value, after recording the operations on the GradientTape.

A function (tf.function) is an efficient way to process tensorts. Tensorflow can execute a tf.Graph, which is an efficient DAG for performing tf.Operations (internal nodes) on tf.Tensors (inputs) to produce a single tf.Tensor (output). Graphs are created from Python code, but don't require the Python runtime and are much more efficient. The object model, per https://www.tensorflow.org/guide/function, is:
* tf.function wraps a Python function, returning a Function object
* A Function manages a cache of ConcreteFunctions and picks the most specific match based on the signature of inputs, or performs tracing to create a tfGraph and wrap it in a ConcreteFunction if there is no match
* A ConcreteFunction wraps a tf.Graph
* A tf.Graph is the raw, language-agnostic, portable representation of a TensorFlow computation
Tracing captures the tensorflow operations into a graph, converts some python code (e.g., numpy operations) into tensorflow first, and discards other operations like "print" statements. By default a Function will execute a graph if it can. A tf.function can also run "eagerly", where it executes the underlying python code rather than the graph.

A Model is a series of Layers. Both Models and Layers are typically implemented as subclasses of tf.Module, which is a container of tf.Variables (trainable parameters) and other tf.Modules (layers).

In practice, one normally uses Keras. The tf.keras.layers.Layer subclasses tf.Module and adds a lot of functionality. That is a topic for another notebook.

In [1]:
import tensorflow as tf
import timeit
from datetime import datetime
from os import listdir, remove

A tensor (tf.constant) is an immutable multi-dimensional array, optimized for GPU processing

In [2]:
# Tensor basics

# A 2x2 matrix
a = tf.constant([[1.0, 2.0], [3.0, 4.0]])

# A 2x1 variable
b = tf.constant([[1.0], [2.0]])

c = a @ b
print (f"{a} @ {b} = {c}")
print (f"Dimensions of c: {c.shape}")
print (f"Type of c: {c.dtype}")

c = tf.Variable(c)


[[1. 2.]
 [3. 4.]] @ [[1.]
 [2.]] = [[ 5.]
 [11.]]
Dimensions of c: (2, 1)
Type of c: <dtype: 'float32'>


A variable (tf.variable) is a mutable tensor

In [3]:
# Variable basics - just like tensors, but can be modified

a = tf.Variable([2.0, 3.0])
print (f"a = {a}")
print (f"type of a: {a.dtype}")

# Reassigning values doesn't change the dtype
a.assign([1, 2])
print (f"a = {a}")
print (f"type of a: {a.dtype}")

# Note: You can't assign a new shape - only modify values

a = <tf.Variable 'Variable:0' shape=(2,) dtype=float32, numpy=array([2., 3.], dtype=float32)>
type of a: <dtype: 'float32'>
a = <tf.Variable 'Variable:0' shape=(2,) dtype=float32, numpy=array([1., 2.], dtype=float32)>
type of a: <dtype: 'float32'>


Autograd computes the gradients of variables automatically; you first record operations on a GradientTape. Autograd then allows you to find the gradient of any of the input tensors with respect to a computed value, after recording the operations on the GradientTape.

In [4]:
# Autograd allows you to compute gradients of input tensorts wrt output tensors automatically

w = tf.Variable(tf.random.normal((3, 2)), name='w')
b = tf.Variable(tf.zeros(2, dtype=tf.float32), name='b')
x = [[1., 2., 3.]]

with tf.GradientTape(persistent=True) as tape:
  y = x @ w + b
  loss = tf.reduce_mean(y**2)

# Note, the 2nd parameter can be an iterable (list, dictionary, etc), and the gradients will be computed for each element
# and returned as the same iterable type
[dl_dw, dl_db] = tape.gradient(loss, [w, b])

print (f"dl_dw = {dl_dw}")
print (f"dl_db = {dl_db}")

# Note: By default, only tf.Variable objects are recorded in the tape. To record tensors, you must use tape.watch()
# You can mark a tf.Variable as untrainable by setting trainable=False
x0 = tf.Variable(3.0, name='x0')                  # A trainable variable
x1 = tf.Variable(3.0, name='x1', trainable=False) # Not trainable
x2 = tf.Variable(2.0, name='x2') + 1.0            # Not a Variable: A variable + tensor returns a tensor.
x3 = tf.constant(3.0, name='x3')                  # Not a Variable

with tf.GradientTape() as tape:
  y = (x0**2) + (x1**2) + (x2**2)

grad = tape.gradient(y, [x0, x1, x2, x3])

for g in grad:  # So only the first gradient is computed
  print(g)

# Apply the gradient to x0 (the only trainable variable)
print ("x0 before applying gradient:", x0)
optimizer = tf.keras.optimizers.SGD(learning_rate=0.1)
optimizer.apply_gradients(zip(grad, [x0]))
print ("x0 after:", x0)

# Note: In practice, you'll train Modules (more on those later), which specify what is trainable in Module.trainable_variables

dl_dw = [[ 1.8597615 -0.3769089]
 [ 3.719523  -0.7538178]
 [ 5.5792847 -1.1307267]]
dl_db = [ 1.8597615 -0.3769089]
tf.Tensor(6.0, shape=(), dtype=float32)
None
None
None
x0 before applying gradient: <tf.Variable 'x0:0' shape=() dtype=float32, numpy=3.0>
x0 after: <tf.Variable 'x0:0' shape=() dtype=float32, numpy=2.4>


A function (tf.function) is an efficient way to process tensorts. Tensorflow can execute a tf.Graph, which is an efficient DAG for performing tf.Operations (internal nodes) on tf.Tensors (inputs) to produce a single tf.Tensor (output). Graphs are created from Python code, but don't require the Python runtime and are much more efficient. The object model, per https://www.tensorflow.org/guide/function, is:
* tf.function wraps a Python function, returning a Function object
* A Function manages a cache of ConcreteFunctions and picks the most specific match based on the signature of inputs, or performs tracing to create a tfGraph and wrap it in a ConcreteFunction if there is no match
* A ConcreteFunction wraps a tf.Graph
* A tf.Graph is the raw, language-agnostic, portable representation of a TensorFlow computation
Tracing captures the tensorflow operations into a graph, converts some python code (e.g., numpy operations) into tensorflow first, and discards other operations like "print" statements. By default a Function will execute a graph if it can. A tf.function can also run "eagerly", where it executes the underlying python code rather than the graph.

In [5]:
# One way to declare a Function is with a decorator
@tf.function
def add (x, y):
    print (x, y)
    return x+y

# Let's see it execute eagerly and in graph model
print ("This should print output, due to tracing")
a = tf.Variable(1)
b = tf.Variable(2)
af = tf.Variable(1.)
bf = tf.Variable(2.)
assert (add (a, b) == 3)

print ("No output, since the Function now has a cached graph")
assert (add (a, b) == 3)

print ("This should print output, since it's being called with a different signature")
assert (add (af, bf) == 3.0)


This should print output, due to tracing
<tf.Variable 'Variable:0' shape=() dtype=int32> <tf.Variable 'Variable:0' shape=() dtype=int32>
No output, since the Function now has a cached graph
This should print output, since it's being called with a different signature
<tf.Variable 'Variable:0' shape=() dtype=float32> <tf.Variable 'Variable:0' shape=() dtype=float32>


In [6]:
# Graph execution is much faster if there are many small functions, like below
# Less so if there are few large functions
x = tf.random.uniform(shape=[10, 10], minval=-1, maxval=2, dtype=tf.dtypes.int32)

def power(x, y):
  result = tf.eye(10, dtype=tf.dtypes.int32)
  for _ in range(y):
    result = tf.matmul(x, result)
  return result

print("Regular function execution:", timeit.timeit(lambda: power(x, 100), number=1000), "seconds")

# You can force a Function to execute eagerly for test purposes
power_as_graph = tf.function(power)
print("tf.function execution:", timeit.timeit(lambda: power_as_graph(x, 100), number=1000), "seconds")

# You can force a Function to execute eagerly for test purposes
tf.config.run_functions_eagerly(True)
print("tf.function executing eagerly:", timeit.timeit(lambda: power_as_graph(x, 100), number=1000), "seconds")
tf.config.run_functions_eagerly(False)

Regular function execution: 4.442750700000033 seconds
tf.function execution: 0.5394228999998631 seconds
tf.function executing eagerly: 4.387751000000208 seconds



A Model is a series of Layers. Both Models and Layers are typically implemented as subclasses of tf.Module, which is a container of tf.Variables (trainable parameters) and other tf.Modules (layers).
Models have the following common properties:
* submodules - collection of submodules
* variables/trainable_variables/non_trainable_variables - uses reflection, so not very performant
Variables can be marked trainable or non-trainable when declared, as well as via the Model.trainable_variables property

If the __call__ method is a tf.Function, then the model will generate tf.Graphs for faster execution and can be saved and run without Python.

In [7]:
class DenseLayer(tf.Module):
  def __init__(self, input_dim, output_dim, name=None):
    super().__init__(name=name)
    self.w = tf.Variable(
      tf.random.normal([input_dim, output_dim]), name='w')
    self.b = tf.Variable(tf.zeros([output_dim]), name='b', dtype=tf.float32)
    self.u = tf.Variable(tf.zeros([output_dim]), name='u', trainable=False, dtype=tf.float32) # Just to show how to make a non-trainable variable
  def __call__(self, x):
    y = tf.matmul(x, self.w) + self.b + self.u
    return tf.nn.relu(y)
  
d = DenseLayer(input_dim=3, output_dim=2)
d(tf.ones([1, 3]))
print (d.trainable_variables)
print (d.non_trainable_variables)

class MySequentialModule(tf.Module):
  def __init__(self, name=None):
    super().__init__(name=name)

    self.dense_1 = DenseLayer(input_dim=3, output_dim=3)
    self.dense_2 = DenseLayer(input_dim=3, output_dim=2)

  @ tf.function
  def __call__(self, x):
    x = self.dense_1(x)
    return self.dense_2(x)

# You have made a model with two layers
my_model = MySequentialModule(name="the_model")

# Call it, with random results
print("Model results:", my_model(tf.constant([[2.0, 2.0, 2.0]])))

print("Submodules:", my_model.submodules)

for var in my_model.variables:
  print(var, "\n")

(<tf.Variable 'b:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)>, <tf.Variable 'w:0' shape=(3, 2) dtype=float32, numpy=
array([[ 0.11862059, -0.6205737 ],
       [ 0.47175336, -0.7345508 ],
       [-0.24489878, -0.2893617 ]], dtype=float32)>)
(<tf.Variable 'u:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)>,)
Model results: tf.Tensor([[0.69631386 0.        ]], shape=(1, 2), dtype=float32)
Submodules: (<__main__.DenseLayer object at 0x000002253B661960>, <__main__.DenseLayer object at 0x000002253B663C10>)
<tf.Variable 'b:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)> 

<tf.Variable 'u:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)> 

<tf.Variable 'w:0' shape=(3, 3) dtype=float32, numpy=
array([[ 1.8356816, -1.3086845, -0.7664374],
       [ 1.0469161,  0.8300276,  1.0368861],
       [-0.5955064,  3.1537387, -1.3027871]], dtype=float32)> 

<tf.Variable 'b:0' shape=(2,) dtype=float32, numpy=array([0., 0.], d

Modules can be checkpointed. Checkpointing allows you to save the weights so you can resume training later. Checkpointing consist of two kinds of files: the data itself and an index file for metadata. The index file keeps track of what is actually saved and the numbering of checkpoints, while the checkpoint data contains the variable values and their attribute lookup paths.

In [11]:
print("Original model results:", my_model(tf.constant([[1.0, 2.0, 3.0]])))
chkp_path = "data/models/my_checkpoint"
checkpoint = tf.train.Checkpoint(model=my_model)
checkpoint.write(chkp_path)

# Inspect the checkpoint
tf.train.list_variables(chkp_path)

# When you load the model back in, you overwrite the values in the existing model
new_model = MySequentialModule(name="restored_model")
new_checkpoint = tf.train.Checkpoint(model=new_model)
new_checkpoint.restore(chkp_path)

# Should be the same result as above
print ("Restored model results:", new_model(tf.constant([[1.0, 2.0, 3.0]])))

Original model results: tf.Tensor([[2.3431063 0.       ]], shape=(1, 2), dtype=float32)
Restored model results: tf.Tensor([[2.3431063 0.       ]], shape=(1, 2), dtype=float32)


Models can be saved and loaded

In [9]:
# Since __call__ has been defined as a tf.function, it will generate a graph we can visualize within a TensorBoard summary

# First set up logging after clearing prior logs
logdir = "logs/tensorboard_test"
for file in listdir(logdir):
  remove(logdir + "/" + file)
writer = tf.summary.create_file_writer(logdir)

# Create a new model to get a fresh trace, otherwise the summary will not see the graph.
new_model = MySequentialModule()

# Bracket the function call with tf.summary.trace_on() and tf.summary.trace_export().
tf.summary.trace_on(graph=True)
z = print(new_model(tf.constant([[2.0, 2.0, 2.0]])))
with writer.as_default():
  tf.summary.trace_export(
      name="my_func_trace",
      step=0,
      profiler_outdir=logdir)

# View the output in TensorBoard
%load_ext tensorboard
%tensorboard --logdir logs/tensorboard_test

tf.Tensor([[0.      4.12147]], shape=(1, 2), dtype=float32)


Reusing TensorBoard on port 6006 (pid 9580), started 0:31:11 ago. (Use '!kill 9580' to kill it.)

A SavedModel is the recommended way of sharing trained models. A SavedModel contains both a collection of functions and a collection of weights

In [10]:
model_path = "data/models/saved_model"
tf.saved_model.save(my_model, model_path)

# Enumerate all files in the saved_model directory
print ("Files in the saved_model folder:", listdir(model_path))

# Load the model back in
restored_saved_model = tf.saved_model.load(model_path)

# Verify the restored model gives the same results
print ("Original model results:", my_model(tf.constant([[1.0, 2.0, 3.0]])))
print ("Restored model results:", restored_saved_model(tf.constant([[1.0, 2.0, 3.0]])))


INFO:tensorflow:Assets written to: data/models/saved_model\assets
Files in the saved_model folder: ['assets', 'fingerprint.pb', 'saved_model.pb', 'variables']
Original model results: tf.Tensor([[2.3431063 0.       ]], shape=(1, 2), dtype=float32)
Restored model results: tf.Tensor([[2.3431063 0.       ]], shape=(1, 2), dtype=float32)
